In [1]:
import isort
import sys
import platform
import importlib
from types import ModuleType

In [2]:
# module_names = isort.stdlibs.py38.stdlib

In [3]:
def get_stdlib_packages():
    if sys.version_info.minor == 10:
        module_names = sys.stdlib_module_names
    else:
        module_names = isort.stdlibs.py38.stdlib

    external_packages = set()
    for name in module_names:
        if name[0] == '_' or name == 'this' or name == 'antigravity':
            continue
        external_packages.add(name)
    return external_packages

In [4]:
def task1():
    external_packages = get_stdlib_packages()
    v = sys.version_info
    print(f'Python {v.major}.{v.minor}.{v.micro} on {platform.system()} {platform.release()}')
    print(f'Stdlib contains {len(external_packages)} external modules and packages: ')
    print(external_packages)

In [5]:
task1()

Python 3.8.10 on Linux 5.15.0-50-generic
Stdlib contains 212 external modules and packages: 
{'multiprocessing', 'bdb', 'filecmp', 'zipimport', 'cmd', 'tracemalloc', 'keyword', 'ntpath', 'sre', 'argparse', 'operator', 'wave', 'socketserver', 'selectors', 'ssl', 'token', 'atexit', 'turtle', 'parser', 'pathlib', 'curses', 'modulefinder', 'ossaudiodev', 'venv', 'shutil', 'tarfile', 'cgitb', 'os', 'nntplib', 'cgi', 'lzma', 'base64', 'secrets', 'py_compile', 'syslog', 'profile', 'plistlib', 'smtpd', 'pty', 'pickle', 'zipapp', 'http', 'pstats', 'pickletools', 'pyclbr', 'rlcompleter', 'ipaddress', 'tabnanny', 'hashlib', 'tokenize', 'errno', 'quopri', 'xdrlib', 'termios', 'contextlib', 'configparser', 'sys', 'warnings', 'subprocess', 'tempfile', 'cmath', 'mailcap', 'codeop', 'ensurepip', 'symbol', 'unicodedata', 'cProfile', 'fileinput', 'lib2to3', 'readline', 'numbers', 'fractions', 'bz2', 'gzip', 'weakref', 'tkinter', 'types', 'array', 'msvcrt', 'posix', 'gettext', 'site', 'xml', 'codecs', 'f

In [6]:
def get_real_packages(package_names):
    real_modules = set()
    not_importable_modules = set()
    for name in package_names:
        try:
            importlib.import_module(name)
            # del sys.modules[name]
            real_modules.add(name)
        except:
            not_importable_modules.add(name)
    return real_modules, not_importable_modules

In [7]:
def get_real():
    external_packages = get_stdlib_packages()
    return get_real_packages(external_packages)

In [8]:
def task2():
    v = sys.version_info
    real_modules, not_importable_modules = get_real()
    print(f'These StdLib packages on Python-{v.major}.{v.minor}.{v.micro}/{platform.system()} {platform.release()} '
          f'are not importable:')
    print(not_importable_modules)

    for name in real_modules:
        del sys.modules[name]

In [9]:
task2()

These StdLib packages on Python-3.8.10/Linux 5.15.0-50-generic are not importable:
{'turtledemo', 'msvcrt', 'msilib', 'winsound', 'ensurepip', 'sre', 'tkinter', 'winreg', 'turtle'}


In [10]:
def module_dependency(module_names, name):
    if name not in module_names:
        raise Exception(f'{name} is not importable module')
    dp_names = set()

    try:
        importlib.import_module(name)
    except:
        print(f'err name: {name}')
    for key, val in vars(sys.modules[name]).items():
        if isinstance(val, ModuleType):
            md_name = val.__name__

            try:
                index = md_name.index(".")
                md_name = md_name[0:index]
            except:
                pass

            dp_names.add(md_name)
            # print(f'key: {key}, type: {type(val)}, val: {md_name}, module: {md_name}')

    # del sys.modules[name]

    return dp_names

In [11]:
def core_modules(real_modules):
    # real_modules, _ = get_real()
    core_module_names = set()
    for r_name in real_modules:
        dp_names = module_dependency(real_modules, r_name)
        if len(dp_names) == 0:
            core_module_names.add(r_name)

    return core_module_names


In [12]:
def most_dependent_modules(real_modules):
    dp_dict = dict((name, 0) for name in real_modules)
    for md_name in real_modules:
        count = len(module_dependency(real_modules, md_name))
        dp_dict[md_name] = count

    sorted_dp_dict = dict(sorted(dp_dict.items(), key=lambda item: item[1], reverse=True))
    i = 0
    most_dependent_module_names = dict()
    for k, v in sorted_dp_dict.items():
        if i > 5:
            break
        most_dependent_module_names[k] = v
        i = i + 1

    return most_dependent_module_names



In [13]:
def task3():
    real_modules, not_importable_modules = get_real()
    # print(f'real modules: {real_modules}')
    core_module_names = core_modules(real_modules)

    dp_dict = most_dependent_modules(real_modules)

    print(f'The following StdLib packages are most dependent:')
    for k, v in dp_dict.items():
        print(f'{k}: {v}')
    print(f'The {len(core_module_names)} core packages are:')
    print(core_module_names)

    for name in real_modules:
        del sys.modules[name]

In [14]:
task3()

The following StdLib packages are most dependent:
pathlib: 8
gzip: 8
shutil: 7
zipimport: 6
socket: 6
re: 6
The 67 core packages are:
{'termios', 'sys', 'abc', 'nis', 'binascii', 'urllib', 'cmath', 'select', 'stat', 'concurrent', 'resource', 'copy', 'grp', 'sre_parse', 'keyword', 'tty', 'symbol', 'unittest', 'unicodedata', 'zlib', 'spwd', 'operator', 'gc', 'faulthandler', 'functools', 'lib2to3', 'decimal', 'readline', 'itertools', 'contextvars', 'numbers', 'ast', 'token', 'atexit', 'email', 'types', 'copyreg', 'parser', 'pwd', 'imghdr', 'array', 'posix', 'xml', 'difflib', 'bisect', 'fcntl', 'ossaudiodev', 'math', 'builtins', 'sre_constants', 'colorsys', 'heapq', 'audioop', 'wsgiref', 'stringprep', 'syslog', 'mmap', 'sndhdr', 'struct', 'xmlrpc', 'http', 'chunk', 'test', 'marshal', 'time', 'errno', 'quopri'}
